In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import make_balance, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize, simplex_proj

from time import time, sleep

from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
%load_ext line_profiler

Loading BokehJS ...

In [17]:
env.action_df

,BTC,ETH,LTC,ETC,XRP,XMR,DASH,BTC,online
2017-06-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Environment setup
obs_steps = 233
period = 240
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP", "USDT_XMR", "USDT_ETC", "USDT_DASH"]
init_funds = make_balance(crypto=1 / len(pairs), fiat=0.0, pairs=pairs)

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, load_dir='./data', pairs=pairs, portfolio=init_funds)
# tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
#                        start=datetime.timestamp(datetime.utcnow() - timedelta(days=200)))
tapi.load_data('/train')

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'BTC'
obs = env.reset()

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!

ERROR:backtest_env:[<property object at 0x7fe134da8ea8>]
No valid value on portfolio dataframe.



600 intervals, or 100 days of data at 240 minutes period downloaded.


KeyError: 'USDT'

In [10]:
# Training run
# Training params
nb_steps = 300
batch_size = 1
nb_max_episode_steps = 33

agent = apriori.MomentumTrader(ma_span=[3,50], std_span=50, activation=simplex_proj)

hp = {
    'ma1': [2, env.obs_steps],
    'ma2': [2, env.obs_steps],
    'std_span': [2, env.obs_steps],
    'alpha_up': [1e-4, 5e-1],
    'alpha_down': [1e-4, 5e-1]
    }

search_space = {'mean_type':{'simple': hp,
                             'exp': hp,
                             'kama': hp
                             }
                }
constrains = [lambda mean_type, ma1, ma2, std_span, alpha_up, alpha_down: ma1 < ma2]

opt_params, info = agent.fit(env, nb_steps, batch_size, search_space, constrains, nb_max_episode_steps=nb_max_episode_steps)
print("\n", opt_params,"\n", env.status)

# Previous found params:
# {'mean_type': 'kama', 'alpha_down': 0.4128425807825884, 'alpha_up': 0.37140222586733046, 'ma1': 133, 'ma2': 234, 'std_span': 39}

agent.test(env, verbose=True)
env.plot_results();

Optimizing model for 300 steps with batch size 1...
Optimization step 230/300, step reward: 1.177576253375126, ETC: 0 days 00:05:20.874667                      
 {'mean_type': 'kama', 'alpha_down': 0.3019641189474816, 'alpha_up': 0.26418681867001537, 'ma1': 91.02373046874999, 'ma2': 190.56670766815745, 'std_span': 54.92998046874999} 
 {'OOD': False, 'Error': False, 'ValueError': False, 'ActionError': False}
>> step 305/305, 100 % done, Cumulative Reward: 1.639269584638691, ETC: 0 days 00:00:00, Samples/s: 7.8089                               
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 0.047388
Portifolio / Benchmark Sharpe ratio:      1.417397 / 1.243485
Portifolio / Benchmark Omega ratio:       1.286010 / 1.247146
Portifolio / Benchmark max drawdown:      -4644205.776586 / -385152143.978367


In [11]:
# Validation run
# tapi.download_data(end=datetime.timestamp(datetime.now() - timedelta(days=50)),
#                        start=datetime.timestamp(datetime.now() - timedelta(days=100)))
env.tapi.load_data('/eval')
agent.test(env, verbose=True)
env.plot_results();

>> step 305/305, 100 % done, Cumulative Reward: 1.6486242875230972, ETC: 0 days 00:00:00, Samples/s: 7.2565                              
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 0.102160
Portifolio / Benchmark Sharpe ratio:      1.677066 / 0.382866
Portifolio / Benchmark Omega ratio:       1.333454 / 1.067359
Portifolio / Benchmark max drawdown:      -20357.765419 / -362843.106932


In [12]:
# Test run
# tapi.download_data(end=datetime.timestamp(datetime.now()),
#                        start=datetime.timestamp(datetime.now() - timedelta(days=50)))
env.tapi.load_data('/test')
agent.test(env, verbose=True)
env.plot_results();

>> step 305/305, 100 % done, Cumulative Reward: 1.639269584638691, ETC: 0 days 00:00:00, Samples/s: 7.7300                               
################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 0.047388
Portifolio / Benchmark Sharpe ratio:      1.417397 / 1.243485
Portifolio / Benchmark Omega ratio:       1.286010 / 1.247146
Portifolio / Benchmark max drawdown:      -4644205.776586 / -385152143.978367
